In [1]:
import pandas as pd
import os
import numpy as np

import torchvision
import torch
import torchvision.transforms as tf
import torch.utils.data.dataset
from tqdm import tqdm

from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from seg_utils_cerebellum import Psdataset_seg, FlipImage, Normalize, Compose, train_classifier, val_classifier

In [4]:
train_transformation = Compose([FlipImage(axis=1, prob=0.5), 
                                   FlipImage(axis=0, prob=0.5), 
                                   Normalize()])
test_transformation = Compose([Normalize()])

In [5]:
train_transformation

Compose(
)

In [6]:
train_dataset = Psdataset_seg("segmentation_cerebellum.csv",cerebellum=True, transforms=train_transformation)
val_dataset = Psdataset_seg("segmentation_cerebellum.csv", train=False, transforms=test_transformation)

In [7]:
train_dataset

In [8]:
x, t = train_dataset[10]
print(x.shape, t.shape)

torch.Size([1, 256, 256]) torch.Size([1, 256, 256])


In [9]:
v, vt = val_dataset[9]
print(v.shape, vt.shape)

torch.Size([1, 256, 256]) torch.Size([1, 256, 256])


In [10]:
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_dataloaders = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=4)

In [11]:
for img, target in train_dataloaders:
    print(img.shape, target.shape)
    break

torch.Size([32, 1, 256, 256]) torch.Size([32, 1, 256, 256])


In [12]:
import segmentation_models_pytorch as smp

In [13]:
net = smp.Unet('resnet34', classes=1, activation='sigmoid')
net.encoder.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [14]:
# Optimizing and creating the loss function #
optim = torch.optim.Adam(net.parameters(), lr=0.01)
loss = smp.utils.losses.DiceLoss()

In [15]:
cuda_avail = torch.cuda.is_available()
if cuda_avail:
    net = net.cuda()

In [16]:
x = net(img.cuda())
x.shape

torch.Size([32, 1, 256, 256])

In [17]:
cuda_avail

True

In [18]:
epochs = 10
for ep in range(epochs):
    train_loss = train_classifier(net, train_dataloaders, loss, optim, cuda_avail)
    val_loss = val_classifier(net, val_dataloaders, loss, cuda_avail)
    train_loss = val_classifier(net, train_dataloaders, loss, cuda_avail)
    print(train_loss, val_loss)

  0%|          | 0/62 [00:00<?, ?it/s]

0.25356582 0.8704235


  0%|          | 0/62 [00:00<?, ?it/s]

0.17837705 0.2712851


  0%|          | 0/62 [00:00<?, ?it/s]

...

0.04981791 0.06558386


  0%|          | 0/62 [00:00<?, ?it/s]

0.056738123 0.05839597


100%|██████████| 62/62 [00:11<00:00,  5.21it/s]


0.055763993 0.06553906


In [20]:
torch.save(net, '/home/shubham/Documents/in-meds-brain-segmentation/pytorch/train_1.h5')